In [531]:
import matplotlib.pyplot as plt
from matplotlib.patches import Arc
import random
import math
import sys
import numpy as np
# %matplotlib inline
%matplotlib tk

In [532]:
# Parametros 
TAMANHO_DO_PLANO = 1000
QUANT_OBSTACULOS = 30
RAIO = 50

PONTO_INICIO = (0,0)
PONTO_FIM = (TAMANHO_DO_PLANO,TAMANHO_DO_PLANO)

mapa_obstaculos = []
arestas_globais = []



In [533]:
def encerrar_com_mensagem(ax,mensagem, cor='red'):
    ax.text(0.5, 0.5, mensagem, ha='center', va='center', fontsize=16, color=cor, transform=ax.transAxes)
    plt.draw()
    plt.show(block=True)
    sys.exit()



In [534]:
def calcular_distancia(ponto1, ponto2): 
    return math.sqrt((ponto1[0] - ponto2[0])**2 + (ponto1[1] - ponto2[1])**2) #  é a formula Euclidiana basicamente,  melhor forma pelo que eu vi ( a outra que eu tenti ainda tinha sobreposição na diagonal )



In [535]:
def verifica_intervalo(ponto_inserido,ponto_verifica):
    distancia = calcular_distancia(ponto_inserido,ponto_verifica)
    return ((distancia > (RAIO * 2)) | (distancia < 0))



In [536]:
def calcular_laterais(ponto_centro):
    ponto_cima = (ponto_centro[0] + RAIO,ponto_centro[1])
    ponto_baixo = (ponto_centro[0] - RAIO,ponto_centro[1])
    ponto_direito = (ponto_centro[0] ,ponto_centro[1] + RAIO)
    ponto_esquerdo = (ponto_centro[0] ,ponto_centro[1] - RAIO)
    return  ponto_cima,ponto_baixo,ponto_direito,ponto_esquerdo



In [537]:
def calcular_distancia_reta(r_a, r_b, r_c, c_x, c_y):
    cima = abs((r_a * c_x) + (r_b * c_y) + r_c)
    baixo = math.sqrt((r_a ** 2) + (r_b ** 2))
    return cima / baixo




In [538]:
def gerar_obstaculo_existente(novo_centro):
    if calcular_distancia(novo_centro, PONTO_INICIO) < RAIO:
        return True
    if calcular_distancia(novo_centro, PONTO_FIM) < RAIO:
        return True
    if any(not verifica_intervalo(novo_centro, obstaculo[0]) for obstaculo in mapa_obstaculos):
        return True
    return False



In [539]:
def posicionar_obstaculos():
    for _ in range(QUANT_OBSTACULOS):
        tentativas = 0
        while True:
            tentativas += 1
            if tentativas >= 1000:
                print(f"Não há mais espaço! Foram posicionados {len(mapa_obstaculos)} obstáculos.")
                return  # Sai do posicionamento, mas mantém os já posicionados
            x = random.uniform(RAIO, TAMANHO_DO_PLANO - RAIO)
            y = random.uniform(RAIO, TAMANHO_DO_PLANO - RAIO)
            novo_centro = (x, y)
            if gerar_obstaculo_existente(novo_centro):
                continue
            mapa_obstaculos.append((novo_centro, calcular_laterais(novo_centro)))
            break



In [540]:
def reta_livre_de_obstaculos(A, B, mapa_obstaculos, raio):
    for obstaculo in mapa_obstaculos:
        centro = obstaculo[0]
        laterais = obstaculo[1]
        
        # 1. Se A e B são laterais do mesmo círculo, bloqueia
        if A in laterais and B in laterais:
            return False
            
        # 2. Coeficientes da reta AB: ax + by + c = 0
        r_a = B[1] - A[1]
        r_b = A[0] - B[0] 
        r_c = B[1] * (B[0] - A[0]) - (B[1] - A[1]) * B[0]

        # 3. Distância do centro do círculo à reta
        dist_reta = calcular_distancia_reta(r_a, r_b, r_c, centro[0], centro[1])
        
        # 4. Se a distância for menor que o raio, há interseção
        if dist_reta < raio:
            # 5. Verifica se a interseção está no segmento AB (não na reta infinita)
            # Calcula o ponto mais próximo do centro na reta AB
            denominador = r_a * r_a + r_b * r_b
            if denominador != 0:
                px = (r_b * (r_b * centro[0] - r_a * centro[1]) - r_a * r_c) / denominador
                py = (r_a * (-r_b * centro[0] + r_a * centro[1]) - r_b * r_c) / denominador
                
                # Verifica se o ponto projetado está dentro do segmento AB
                min_x, max_x = min(A[0], B[0]), max(A[0], B[0])
                min_y, max_y = min(A[1], B[1]), max(A[1], B[1])
                
                if min_x <= px <= max_x and min_y <= py <= max_y:
                    return False
    return True



In [541]:
def gerar_arestas():
    # Gera todas as arestas válidas e as adiciona aos obstáculos
    # Formato: [ponto_centro, [pontos_laterais], [arestas]]
    
    # Primeiro, inicializa a lista de arestas para cada obstáculo
    for i, obstaculo in enumerate(mapa_obstaculos):
        if len(obstaculo) == 2:  # Se ainda não tem arestas
            mapa_obstaculos[i] = (obstaculo[0], obstaculo[1], [])
    
    # Arestas do início aos pontos laterais
    for obstaculo in mapa_obstaculos:
        for ponto_lateral in obstaculo[1]:
            if reta_livre_de_obstaculos(PONTO_INICIO, ponto_lateral, mapa_obstaculos, RAIO):
                aresta = (PONTO_INICIO, ponto_lateral, 'blue', '--')
                arestas_globais.append(aresta)
    
    # Arestas do fim aos pontos laterais
    for obstaculo in mapa_obstaculos:
        for ponto_lateral in obstaculo[1]:
            if reta_livre_de_obstaculos(PONTO_FIM, ponto_lateral, mapa_obstaculos, RAIO):
                aresta = (PONTO_FIM, ponto_lateral, 'red', '--')
                arestas_globais.append(aresta)
    
    # Arestas entre pontos laterais (evita duplicação)
    for i, obstaculo_a in enumerate(mapa_obstaculos):
        for j, obstaculo_b in enumerate(mapa_obstaculos):
            if i >= j:  # Evita duplicação (só processa i < j)
                continue
            for ponto_lateral_a in obstaculo_a[1]:
                for ponto_lateral_b in obstaculo_b[1]:
                    if reta_livre_de_obstaculos(ponto_lateral_a, ponto_lateral_b, mapa_obstaculos, RAIO):
                        aresta = (ponto_lateral_a, ponto_lateral_b, 'green', '-')
                        # Adiciona à lista do primeiro obstáculo
                        mapa_obstaculos[i] = (mapa_obstaculos[i][0], mapa_obstaculos[i][1], mapa_obstaculos[i][2] + [aresta])
    
    # Verifica caminho direto início-fim
    if reta_livre_de_obstaculos(PONTO_INICIO, PONTO_FIM, mapa_obstaculos, RAIO):
        aresta = (PONTO_INICIO, PONTO_FIM, 'purple', '-')
        arestas_globais.append(aresta)



In [542]:
def caminho_ini_fim():
    """
        Percorre o mapa de obstaculos entre os pontos dos obstaculos, mas caso não tenha obstaculos, faz a linha direta, utilizando o conceito de busca por profundidade    
    """
    
    if len(mapa_obstaculos) == 0:
        return [PONTO_INICIO, PONTO_FIM]
    
    # Implementação de busca por profundidade (DFS) para encontrar um caminho
    stack = [(PONTO_INICIO, [PONTO_INICIO])]
    visited = set()
    
    while stack:
        current, path = stack.pop()
        
        if current == PONTO_FIM:
            return path
        
        if current in visited:
            continue
        visited.add(current)
        
        # Adiciona vizinhos (pontos laterais dos obstáculos conectados)
        for obstaculo in mapa_obstaculos:
            for ponto_lateral in obstaculo[1]:
                if reta_livre_de_obstaculos(current, ponto_lateral, mapa_obstaculos, RAIO):
                    stack.append((ponto_lateral, path + [ponto_lateral]))
    
    return None  # Se não encontrar caminho




In [543]:
def inicializar_plot():
    fig, ax = plt.subplots()
    ax.set_xlim(0, TAMANHO_DO_PLANO)
    ax.set_ylim(0, TAMANHO_DO_PLANO)
    ax.set_aspect('equal')
    ax.grid(color='gray', linestyle='--', alpha=0.3)
    ax.axhline(0, color='white', linewidth=0.5)
    ax.axvline(0, color='white', linewidth=0.5)
    ax.plot(PONTO_INICIO[0], PONTO_INICIO[1], 'go', markersize=10, label='Início')
    ax.plot(PONTO_FIM[0], PONTO_FIM[1], 'ro', markersize=10, label='Fim')
    
    # Opção 2: Colocar fora da figura (à direita)
    ax.legend(loc='upper left', bbox_to_anchor=(1.02, 1.0), frameon=True)
    plt.tight_layout()  # Ajusta automaticamente para não cortar
    
    return fig, ax


In [544]:
def plotar_arestas(ax):
    # Plota arestas globais (início/fim)
    for aresta in arestas_globais:
        ponto_a, ponto_b, cor, estilo = aresta
        ax.plot([ponto_a[0], ponto_b[0]], [ponto_a[1], ponto_b[1]], 
                color=cor, linestyle=estilo, linewidth=1)

    # Plota arestas dos obstáculos
    for obstaculo in mapa_obstaculos:
        if len(obstaculo) > 2:  # Se tem arestas
            for aresta in obstaculo[2]:
                ponto_a, ponto_b, cor, estilo = aresta
                ax.plot([ponto_a[0], ponto_b[0]], [ponto_a[1], ponto_b[1]], 
                        color=cor, linestyle=estilo, linewidth=1)

In [545]:
def plotar_obstaculos():
    # Inicializa o plot com início e fim
    _,ax = inicializar_plot()
    
    # Desenha todos os obstáculos
    for obstaculo in mapa_obstaculos:
        circulo = plt.Circle(obstaculo[0], RAIO, color="gray", alpha=0.7)
        for ponto in obstaculo[1]:
            ax.plot(ponto[0], ponto[1], 'o', markersize=2, color="black")
        ax.add_patch(circulo)
    
    plotar_arestas(ax)
    
    plt.title('Mapa de Obstáculos')
    plt.show()



In [546]:
def main():
    posicionar_obstaculos()
    gerar_arestas()
    print(f"Posicionados {len(mapa_obstaculos)} obstáculos")
    plotar_obstaculos()

if __name__ == "__main__":
    main()

Posicionados 30 obstáculos


In [547]:
mapa_obstaculos[0]

((820.1780197620144, 95.79060599850459),
 ((870.1780197620144, 95.79060599850459),
  (770.1780197620144, 95.79060599850459),
  (820.1780197620144, 145.7906059985046),
  (820.1780197620144, 45.79060599850459)),
 [((770.1780197620144, 95.79060599850459),
   (735.9940179708384, 126.6041994534834),
   'green',
   '-'),
  ((820.1780197620144, 145.7906059985046),
   (785.9940179708384, 176.6041994534834),
   'green',
   '-'),
  ((820.1780197620144, 145.7906059985046),
   (286.0780359393548, 895.8023439412658),
   'green',
   '-'),
  ((870.1780197620144, 95.79060599850459),
   (871.2630013309401, 269.63214110629997),
   'green',
   '-'),
  ((820.1780197620144, 145.7906059985046),
   (821.2630013309401, 319.63214110629997),
   'green',
   '-'),
  ((820.1780197620144, 145.7906059985046),
   (871.2630013309401, 269.63214110629997),
   'green',
   '-'),
  ((770.1780197620144, 95.79060599850459),
   (609.6908000036001, 64.26349621333523),
   'green',
   '-'),
  ((770.1780197620144, 95.790605998504

In [548]:
arestas_globais

[((0, 0), (232.04531436573143, 2.1113577153376895), 'blue', '--'),
 ((0, 0), (1.7710112934712683, 407.6716519890643), 'blue', '--'),
 ((0, 0), (51.77101129347127, 357.6716519890643), 'blue', '--'),
 ((0, 0), (289.1486789072892, 331.5511665083218), 'blue', '--'),
 ((0, 0), (128.80060911158245, 339.3265871357303), 'blue', '--'),
 ((0, 0), (178.80060911158245, 289.3265871357303), 'blue', '--'),
 ((0, 0), (211.79654300876945, 742.8682863970604), 'blue', '--'),
 ((0, 0), (364.02923321967523, 444.5462192821156), 'blue', '--'),
 ((0, 0), (257.22584839672544, 736.9769586676098), 'blue', '--'),
 ((0, 0), (213.4383411248029, 601.6825654838126), 'blue', '--'),
 ((0, 0), (69.88325575992592, 52.241138214738946), 'blue', '--'),
 ((0, 0), (119.88325575992592, 2.241138214738946), 'blue', '--'),
 ((1000, 1000), (786.0626714619428, 828.148460507579), 'red', '--'),
 ((1000, 1000), (736.0626714619428, 878.148460507579), 'red', '--'),
 ((1000, 1000), (286.0780359393548, 895.8023439412658), 'red', '--'),
 (

In [549]:
dict_caminhos = {}

# Inicializa todos os pontos laterais no dicionário
for obstaculo in mapa_obstaculos:
    for ponto_lateral in obstaculo[1]:
        dict_caminhos[ponto_lateral] = []

# Adiciona PONTO_INICIO e PONTO_FIM ao dicionário
dict_caminhos[PONTO_INICIO] = []
dict_caminhos[PONTO_FIM] = []

# Processa arestas dos obstáculos (conexões entre pontos laterais de obstáculos diferentes)
for obstaculo in mapa_obstaculos:
    if len(obstaculo) > 2:  # Se tem arestas
        for aresta in obstaculo[2]:
            ponto_a, ponto_b = aresta[0], aresta[1]
            # Adiciona conexão bidirecional
            if ponto_a in dict_caminhos:
                dict_caminhos[ponto_a].append(ponto_b)
                dict_caminhos[ponto_b].append(ponto_a)

# Processa arestas globais (conexões com PONTO_INICIO e PONTO_FIM)
for aresta in arestas_globais:
    ponto_a, ponto_b = aresta[0], aresta[1]
    # Adiciona conexão bidirecional
    if ponto_a in dict_caminhos and ponto_b in dict_caminhos and ponto_a == PONTO_INICIO:
        dict_caminhos[ponto_a].append(ponto_b)
        dict_caminhos[ponto_b].append(ponto_a)
    elif ponto_a in dict_caminhos and ponto_b in dict_caminhos and ponto_a == PONTO_FIM:
        dict_caminhos[ponto_b].append(ponto_a)
        dict_caminhos[ponto_a].append(ponto_b)

In [550]:
for obstaculo in mapa_obstaculos:
    ponto_cima,ponto_baixo,ponto_direito,ponto_esquerdo = obstaculo[1]
    dict_caminhos[ponto_cima].append(ponto_esquerdo)
    dict_caminhos[ponto_esquerdo].append(ponto_cima)
    
    dict_caminhos[ponto_esquerdo].append(ponto_baixo)
    dict_caminhos[ponto_baixo].append(ponto_esquerdo)
    
    dict_caminhos[ponto_baixo].append(ponto_direito)
    dict_caminhos[ponto_direito].append(ponto_baixo)
    
    dict_caminhos[ponto_direito].append(ponto_cima)
    dict_caminhos[ponto_cima].append(ponto_direito)

In [551]:
for key, value in dict_caminhos.items():
    print(f"Reta: {key}, Pontos: {value}")

Reta: (870.1780197620144, 95.79060599850459), Pontos: [(871.2630013309401, 269.63214110629997), (888.4019647510395, 105.3337287265379), (820.1780197620144, 45.79060599850459), (820.1780197620144, 145.7906059985046)]
Reta: (770.1780197620144, 95.79060599850459), Pontos: [(735.9940179708384, 126.6041994534834), (609.6908000036001, 64.26349621333523), (327.0086089218378, 150.09695703544617), (178.80060911158245, 289.3265871357303), (533.4833247459173, 194.37776720240893), (820.1780197620144, 45.79060599850459), (820.1780197620144, 145.7906059985046)]
Reta: (820.1780197620144, 145.7906059985046), Pontos: [(785.9940179708384, 176.6041994534834), (286.0780359393548, 895.8023439412658), (821.2630013309401, 319.63214110629997), (871.2630013309401, 269.63214110629997), (738.5873752002443, 419.41118267299834), (688.5873752002443, 369.41118267299834), (770.1780197620144, 95.79060599850459), (870.1780197620144, 95.79060599850459)]
Reta: (820.1780197620144, 45.79060599850459), Pontos: [(870.1780197

In [552]:
ponto_inicial = PONTO_INICIO
ponto_final = PONTO_FIM
possibilidade_atual = dict_caminhos[ponto_inicial]
print(possibilidade_atual)

def encontrar_possibilidades(ponto_atual, ponto_final, pilha=[]):
    if ponto_atual in pilha:
        return False
    pilha.append(ponto_atual)

    if ponto_atual == ponto_final:
        return True

    for prox_ponto in dict_caminhos.get(ponto_atual, []):
        if encontrar_possibilidades(prox_ponto, ponto_final, pilha):
            return True

    pilha.pop()
    return False

pilha = []
caminho_encontrado = encontrar_possibilidades(ponto_inicial, ponto_final, pilha)

[(232.04531436573143, 2.1113577153376895), (1.7710112934712683, 407.6716519890643), (51.77101129347127, 357.6716519890643), (289.1486789072892, 331.5511665083218), (128.80060911158245, 339.3265871357303), (178.80060911158245, 289.3265871357303), (211.79654300876945, 742.8682863970604), (364.02923321967523, 444.5462192821156), (257.22584839672544, 736.9769586676098), (213.4383411248029, 601.6825654838126), (69.88325575992592, 52.241138214738946), (119.88325575992592, 2.241138214738946)]


In [553]:
def ponto_mesmo_centroide(ponto1, ponto2):
    for obstaculo in mapa_obstaculos:
        centro, laterais = obstaculo[0], obstaculo[1]
        if ponto1 in laterais and ponto2 in laterais:
            return True, centro
    return False, None


In [554]:
fig, ax = inicializar_plot()

# Plota obstáculos e arestas
for obstaculo in mapa_obstaculos:
    circulo = plt.Circle(obstaculo[0], RAIO, color="gray", alpha=0.7)
    for ponto in obstaculo[1]:
        ax.plot(ponto[0], ponto[1], 'o', markersize=2, color="black")
    ax.add_patch(circulo)
plotar_arestas(ax)

# Plota o caminho encontrado
if caminho_encontrado:
    for i in range(len(pilha) - 1):
        ponto1 = pilha[i]
        ponto2 = pilha[i + 1]
        
        mesmo_centro, centroid = ponto_mesmo_centroide(ponto1, ponto2)
        # print(mesmo_centro, centroid)
        if mesmo_centro:
            # Desenha arco entre pontos do mesmo obstáculo
            angulo1 = np.degrees(np.arctan2(ponto1[1] - centroid[1], ponto1[0] - centroid[0]))
            angulo2 = np.degrees(np.arctan2(ponto2[1] - centroid[1], ponto2[0] - centroid[0]))
            arco = Arc(centroid, width=2*RAIO, height=2*RAIO, theta1=angulo2, theta2=angulo1, color="orange", linewidth=2)
            ax.add_patch(arco)
            
            # ax.plot([ponto1[0], ponto2[0]], [ponto1[1], ponto2[1]], 
            #     color='orange', linewidth=3, marker='o')
            
        else:
            # Desenha linha reta entre pontos
            ax.plot([ponto1[0], ponto2[0]], [ponto1[1], ponto2[1]], 
                   color='orange', linewidth=3, marker='o')
    
    ax.legend(loc='upper left', bbox_to_anchor=(1.02, 1.0), frameon=True)

plt.title('Caminho Encontrado')
plt.show()
